In [1]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
import time

# ---------- CONFIG ----------
EXCEL_IN = "video_descriptions_with_links.xlsx"
OUTPUT = "extracted_serving_size.xlsx"
AUTOSAVE_EVERY = 10
HEADLESS = False   # set True once stable
# ----------------------------

df = pd.read_excel(EXCEL_IN)
if "serving_size" not in df.columns:
    df["serving_size"] = ""

# Quick value check
print("Columns in Excel:", df.columns.tolist())
if "recipe_link" not in df.columns:
    raise ValueError("Excel must contain 'recipe_link' column.")


async def extract_serving_for_url(browser, url):
    """Open a fresh page, simulate user interaction, then extract serving size."""
    page = await browser.new_page()
    try:
        await page.goto(url, timeout=60000)

        # 1️ Scroll the page a bit
        await page.evaluate("window.scrollBy(0, 300)")
        await asyncio.sleep(0.5)
        await page.evaluate("window.scrollBy(0, 300)")
        await asyncio.sleep(0.5)

        # 2️ Move the mouse to simulate activity
        await page.mouse.move(100, 100)
        await asyncio.sleep(0.2)
        await page.mouse.move(200, 200)
        await asyncio.sleep(0.2)

        # 3️ Focus the recipe container if exists
        try:
            container = page.locator("div.wprm-recipe-container")
            if await container.count() > 0:
                await container.focus()
                await asyncio.sleep(0.5)
        except:
            pass

        # Wait for input to appear (up to 10s)
        try:
            await page.wait_for_selector("input.wprm-recipe-servings", timeout=10000)
        except:
            pass

        # 1) Try main page
        try:
            element = page.locator("input.wprm-recipe-servings")
            if await element.count() > 0:
                return await element.input_value()
        except:
            pass

        # 2) Try all frames
        for frame in page.frames:
            if frame == page.main_frame:
                continue
            try:
                element = frame.locator("input.wprm-recipe-servings")
                if await element.count() > 0:
                    return await element.input_value()
            except:
                continue

        return "Not found"

    except Exception as e:
        return f"Error: {e}"

    finally:
        await page.close()




async def scrape_all():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=HEADLESS,
                                          args=["--no-sandbox", "--disable-dev-shm-usage"])
        total = len(df)
        for idx, row in df.iterrows():
            display_i = idx + 1

            # Skip if serving_size already filled
            if str(row.get("serving_size")).strip() not in ["", "nan", "NaN"]:
                print(f" [{display_i}/{total}] Skipping (serving_size already present)")
                continue

            link = row.get("recipe_link")
            if pd.isna(link) or str(link).strip() == "":
                print(f"  [{display_i}/{total}] No link; marking and skipping")
                df.at[idx, "serving_size"] = "No link"
                continue

            print(f"🔹 [{display_i}/{total}] Processing: {link}")
            try:
                result = await extract_serving_for_url(browser, str(link))
            except Exception as e:
                result = f"Error: {e}"

            df.at[idx, "serving_size"] = result
            print(f"    => {result}")

            # Autosave
            if display_i % AUTOSAVE_EVERY == 0:
                df.to_excel(OUTPUT, index=False)
                print(f" Autosaved to {OUTPUT} after {display_i} rows")

            # be polite to avoid rate limits
            await asyncio.sleep(1.0)

        await browser.close()

    # final save
    df.to_excel(OUTPUT, index=False)
    print(" Done — final results saved to", OUTPUT)


# Run in Jupyter:
await scrape_all()


Columns in Excel: ['video_id', 'title', 'recipe_link', 'serving_size']
⚠️  [1/2724] No link; marking and skipping
🔹 [2/2724] Processing: https://hebbarskitchen.com/moong-dal-recipe-moong-dal-tadka/
    => 4
🔹 [3/2724] Processing: https://hebbarskitchen.com/bread-masala-recipe-masala-bread/
    => 2
⚠️  [4/2724] No link; marking and skipping
🔹 [5/2724] Processing: https://hebbarskitchen.com/imli-chutney-tamarind-chutney-recipe/
    => 1
🔹 [6/2724] Processing: https://hebbarskitchen.com/coriander-rice-recipe-coriander-pulao/
    => 4
⚠️  [7/2724] No link; marking and skipping
🔹 [8/2724] Processing: https://hebbarskitchen.com/easy-dry-paneer-bhurji-recipe/
    => 3
🔹 [9/2724] Processing: https://hebbarskitchen.com/tomato-upma-recipe-tomato-rava-upma/
    => 3
🔹 [10/2724] Processing: https://hebbarskitchen.com/aloo-bhuna-masala-curry-recipe-potato/
    => 4
💾 Autosaved to extracted_serving_size.xlsx after 10 rows
🔹 [11/2724] Processing: https://hebbarskitchen.com/chilli-garlic-fried-rice-r